In [10]:
import pandas as pd
import numpy as np

## Preprocessing

In [11]:
pd.read_csv("usa_00012.csv.gz")

,YEAR,MULTYEAR,SAMPLE,SERIAL,CBSERIAL,HHWT,CLUSTER,STATEICP,STRATA,GQ,...,EMPSTATD,OCC,IND,UHRSWORK,INCTOT,INCWAGE,MIGRATE1,MIGRATE1D,PWSTATE2,PWCOUNTY
0,2023,2019,202303,1,2019010000088,2.0,2023000000013,41,260001,4,...,30,0,0,0,10619,0,1,10,0,0
1,2023,2019,202303,2,2019010000096,14.0,2023000000023,41,70001,3,...,30,0,0,0,177,0,2,23,0,0
2,2023,2019,202303,3,2019010000153,4.0,2023000000033,41,80001,4,...,10,5240,9160,32,1652,1652,2,24,1,0
3,2023,2019,202303,4,2019010000198,17.0,2023000000043,41,80001,3,...,30,0,0,0,26783,0,2,23,0,0
4,2023,2019,202303,5,2019010000205,11.0,2023000000053,41,280301,3,...,30,0,0,0,0,0,2,23,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15912388,2023,2023,202303,7086486,2023001457972,15.0,2023070864863,68,30056,1,...,10,2320,7860,45,80000,80000,1,10,56,21
15912389,2023,2023,202303,7086486,2023001457972,15.0,2023070864863,68,30056,1,...,30,2310,7860,0,65000,0,1,10,0,0
15912390,2023,2023,202303,7086487,2023001458196,15.0,2023070864873,68,20056,1,...,10,1650,8191,40,75000,75000,1,10,56,0
15912391,2023,2023,202303,7086488,2023001459187,7.0,2023070864883,68,10056,1,...,30,0,0,0,23000,0,1,10,0,0


In [12]:
df = pd.read_csv("usa_00012.csv.gz")

df.to_csv(
    "total_data.csv", 
    index=False,           
    encoding="utf-8",      
    float_format="%.2f"  
)

In [13]:
#features
all_features = df.columns.tolist()
print(all_features)

['YEAR', 'MULTYEAR', 'SAMPLE', 'SERIAL', 'CBSERIAL', 'HHWT', 'CLUSTER', 'STATEICP', 'STRATA', 'GQ', 'NFAMS', 'PERNUM', 'PERWT', 'FAMUNIT', 'FAMSIZE', 'RELATE', 'RELATED', 'SEX', 'AGE', 'RACE', 'RACED', 'ANCESTR1', 'ANCESTR1D', 'EDUC', 'EDUCD', 'SCHLTYPE', 'DEGFIELD', 'DEGFIELDD', 'DEGFIELD2', 'DEGFIELD2D', 'EMPSTAT', 'EMPSTATD', 'OCC', 'IND', 'UHRSWORK', 'INCTOT', 'INCWAGE', 'MIGRATE1', 'MIGRATE1D', 'PWSTATE2', 'PWCOUNTY']


In [14]:
#Filter houshold which fullfil the following conditions:
    # at least 1 child and parent is registered
    # parents have an income > 0
    # child is between 30-40
    # child and parent are both employed

df["parent"] = df["RELATE"].isin([1, 2])  # Head or spouse

df["working_parent"] = (
    df["parent"] & 
    (df["EMPSTAT"].isin([1, 2])) &    # 1 = Employed, 2 = Self-employed
    (df["INCTOT"].between(15000, 300000)) & 
    (df["UHRSWORK"] >= 35)) # Work-hours per week

df["child"] = (df["RELATE"] == 3)  # Biological child

df["working_child"] = (
    df["child"] & 
    (df["AGE"].between(25,35)) &  # Age between 25 and 35
    (df["EMPSTAT"].isin([1, 2])) & # 1 = Employed, 2 = Self-employed
    (df["UHRSWORK"] >= 35) & # > 35 hours of work per week
    (df["INCWAGE"].between(15000,300000))
)

households = df.groupby("SERIAL")

filtered_households = households.filter(
    lambda x: (x["working_child"].any() & 
              x["working_parent"].any())
)

result = filtered_households[filtered_households["working_parent"] | filtered_households["working_child"]]


print(f"Households with valid parent-child pairs: {result['SERIAL'].nunique()}")
print(f"Working parents: {result['working_parent'].sum()}")
print(f"Working children (25-35): {result['working_child'].sum()}")

Households with valid parent-child pairs: 116171
Working parents: 151376
Working children (25-35): 126909


In [15]:
valid_households = result.groupby('SERIAL')

# Print all households and their members
# for household_id, household_data in valid_households:
#     print(f"\n=== Household {household_id} ===")
#     print(household_data[["PERNUM", "AGE", "SEX", "INCTOT", "INCWAGE", "working_parent",'working_child']])
#     print(f"Parents' average income: {household_data.loc[household_data['working_parent'], 'INCTOT'].mean():.2f}")
#     print(f"Adult children's average income: {household_data.loc[household_data['working_child'], 'INCWAGE'].mean():.2f}")

In [16]:
def map_occ_to_category(OCC):
    if 10 <= OCC <= 960: return "Management, Buisness and Financial OCC"
    elif 1005 <= OCC <= 1980: return "Computer, Engineering, Science OCC"
    elif 2001 <= OCC <= 2965: return "Education, Legal, Community Service, Arts, and Media OCC"
    elif 3000 <= OCC <= 3550: return "Healthcare Practitioners and Technical OCC"
    elif 3601 <= OCC <= 4655: return "Service OCC"
    elif 4700 <= OCC <= 4965: return "Sales and Office OCC"
    elif 5000 <= OCC <= 5940: return "Office and Administrative Support OCC"
    elif 6005 <= OCC <= 6130: return "Farming, Fishing, and Forestry OCC"
    elif 6200 <= OCC <= 6950: return "Construction and Extraction OCC"
    elif 7000 <= OCC <= 7640: return "Installation, Maintenance, and Repair OCC"
    elif 7700 <= OCC <= 8990: return "Production, Transportation, and Material Moving OCC"
    elif 9005 <= OCC <= 9760: return "Transportation and Material Moving OCC"
    elif 9800 <= OCC <= 9830: return "Military"

In [20]:
#Select all working students from the valid households
working_children_data = filtered_households[filtered_households['working_child']][
    ["PERNUM", "SERIAL", "SEX", "AGE", "RACE", "RACED","EDUC", "EDUCD",
     "INCTOT","INCWAGE","OCC", "IND","UHRSWORK","PWSTATE2","STATEICP"]
].copy()

parental_data = filtered_households[filtered_households["working_parent"]].groupby("SERIAL").agg(
    parent_income=("INCTOT", "mean"),
    parent_educ=("EDUC", "max")
)

working_children_data = working_children_data.merge(
    parental_data, 
    on="SERIAL", 
    how="left" 
)

data = working_children_data[["AGE","SEX","RACE","EDUC", "OCC","UHRSWORK", "PWSTATE2","INCWAGE", "parent_income", "parent_educ"]]

data = data[(data["SEX"] !=9) &
            (data["PWSTATE2"] !=0) &
            (data["PWSTATE2"] !=99) &
            (data["EDUC"] !=0) &
            (data["EDUC"] != 99) &
            (data["RACE"] !=0) &
            (data["OCC"] !=0) &
            (data["OCC"] !=9920) &
            (data["parent_educ"] != 0)]

data['OCC_CATEGORY'] = data['OCC'].apply(map_occ_to_category)
data = data.drop("OCC", axis = 1)


pd.set_option("expand_frame_repr", False)
print(data)

data.to_csv(
    "filtered_data.csv", 
    index=False,          
    encoding="utf-8",     
    float_format="%.2f"    
)


        AGE  SEX  RACE  EDUC  UHRSWORK  PWSTATE2  INCWAGE  parent_income  parent_educ                                       OCC_CATEGORY
0        25    1     1     6        43         1    29497        72799.0            7              Office and Administrative Support OCC
1        28    1     8     6        40         1    21238        63714.0            6          Installation, Maintenance, and Repair OCC
2        26    2     1    10        43         1    42358        55543.0            6  Education, Legal, Community Service, Arts, and...
3        29    1     1    10        40         1   100290       158105.0           10                 Computer, Engineering, Science OCC
4        28    1     2     7        48         1    47195        27137.0            6  Production, Transportation, and Material Movin...
...     ...  ...   ...   ...       ...       ...      ...            ...          ...                                                ...
126903   25    1     1     6        40   

Total number of individuals: 118215

Working child: 

            EMPSTAT (employment status = 1 or 2 (self-employed))
            AGE: 25-35
            INCWAGE (Income from wage): 15000-300000
            UHRSWORK: >=35

Working child: 

            EMPSTAT (employment status = 1 or 2 (self-employed))
            INCTOT (Total income): 15000-300000
            UHRSWORK: >=35
    
            

In [18]:

# df['adult'] = df['AGE'] > 21
# df['child'] = df['AGE'] <= 21
# df["parent"] = df["RELATE"].isin([1, 2])

# parental_data = df[filtered_households["working_parent"]].groupby("SERIAL").agg(
#     parent_income=("INCTOT", "mean"),
#     parent_educ=("EDUC", "max")
# )

# household_data = df.groupby('SERIAL').agg(
# total_household_income=('INCTOT', 'sum'),
# state=('STATEICP', 'first'),
# num_adults=('adult', 'sum'),
# num_children=('child', 'sum'),
# max_educ_parent=('EDUC', 'max')).reset_index()


In [19]:
import pandas as pd

pd.set_option("expand_frame_repr", False)
df = pd.read_csv("total_data.csv")

print(df)


          YEAR  MULTYEAR  SAMPLE   SERIAL       CBSERIAL  HHWT        CLUSTER  STATEICP  STRATA  GQ  ...  EMPSTATD   OCC   IND  UHRSWORK  INCTOT  INCWAGE  MIGRATE1  MIGRATE1D  PWSTATE2  PWCOUNTY
0         2023      2019  202303        1  2019010000088   2.0  2023000000013        41  260001   4  ...        30     0     0         0   10619        0         1         10         0         0
1         2023      2019  202303        2  2019010000096  14.0  2023000000023        41   70001   3  ...        30     0     0         0     177        0         2         23         0         0
2         2023      2019  202303        3  2019010000153   4.0  2023000000033        41   80001   4  ...        10  5240  9160        32    1652     1652         2         24         1         0
3         2023      2019  202303        4  2019010000198  17.0  2023000000043        41   80001   3  ...        30     0     0         0   26783        0         2         23         0         0
4         2023      2019 